# Apply Warp Transform

## Library Imports

In [1]:
%%time
# Standard Library Imports
import requests
import zipfile
import os

# Third Party Imports
import tifffile
import ants
import numpy as np

# Local Imports
import clearex


CPU times: user 1.78 s, sys: 551 ms, total: 2.33 s
Wall time: 5.05 s


## Download Example Data

In [2]:
%%time
url = "https://cloud.biohpc.swmed.edu/index.php/s/7y7QHszBiJtqgQE/download"
output_path = "downloaded_data.zip"

with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(output_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
print(f"Downloaded to {output_path}")

with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall("downloaded_data")
print(f"Extracted {output_path} directory.")

os.remove(output_path)

Downloaded to downloaded_data.zip
Extracted downloaded_data.zip directory.
CPU times: user 780 ms, sys: 2.3 s, total: 3.08 s
Wall time: 32.3 s


## Specify Paths

In [3]:
%%time

output_path = output_path.split(".")[0]

fixed_path = os.path.join(
    output_path,
    "clearex",
    "cropped_fixed.tif")

moving_path = os.path.join(
    output_path,
    "clearex",
    "cropped_moving.tif")

transform_path = os.path.join(
    output_path,
    "clearex",
    "GenericAffine.mat")

transform_list = [transform_path]

registered_path = os.path.join(
    output_path,
    "clearex",
    "registered.tif")

CPU times: user 10 μs, sys: 0 ns, total: 10 μs
Wall time: 13.1 μs


## Load Images and Transform

In [4]:
%%time
fixed = tifffile.imread(fixed_path)
fixed = ants.from_numpy(fixed)

moving = tifffile.imread(moving_path)
min_value = moving.min()
max_value = moving.max()
moving = ants.from_numpy(moving)


CPU times: user 157 ms, sys: 66.9 ms, total: 224 ms
Wall time: 251 ms


## Apply Transform

In [5]:
%%time
warped_image = ants.apply_transforms(
    fixed=fixed,
    moving=moving,
    transformlist=transform_list,
    interpolator='linear'
)

CPU times: user 473 ms, sys: 1.16 s, total: 1.64 s
Wall time: 387 ms


## Get Information About the Transformed Image

In [6]:
%%time
warped_image

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 2.86 μs


ANTsImage (RAI)
	 Pixel Type : unsigned int (uint32)
	 Components : 1
	 Dimensions : (925, 163, 306)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

## Export the Image

In [ ]:
warped_image = warped_image.numpy().astype(np.uint16)

# Export the transformed and cropped image
clearex.registration.common.export_tiff(
    image=warped_image,
    min_value=min_value,
    max_value=max_value,
    data_path=registered_path)

In [9]:
print(f"Data exported to {registered_path}")

Data exported to downloaded_data/clearex/registered.tif
